In [2]:
import os
import re
from collections import OrderedDict
import numpy as np

database = '/home/ruidong/Documents/Research/Cancer_Genomics/Data/Databases/Gene_ID_conversion/'
os.chdir('/home/ruidong/Documents/Research/share/methylation/')

In [23]:
fh = open('methylation.beta_value27_and_traits.paired.TCGA-STAD.txt', 'wt')

with open('methylation.beta_value27_and_traits.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            sortedList = sorted(lst)
            samples = sortedList[1:sortedList.index('TCGA-BR-4376-11A')+1]
            #samples = [sam for sam in samples if sam.endswith('11A')] + [sam for sam in samples if sam.endswith('01A')]
            indexSam = [lst.index(sam) for sam in samples]
            
            #print (indexSam)
            #for sam in samples:
            #    methyDa[sam] = []
                
        fh.write(lst[0]+'\t'+'\t'.join([lst[i] for i in indexSam])+'\n')
        

fh.close()
        
        

In [26]:
############# input for survival analysis

def therapy_info(x):
    x = np.array(x)
    y = ['NA']*x.shape[1]
    for i in range(x.shape[1]):
        z = x[:,i]
        if 'YES' in z or 'Chemotherapy' in z:
            y[i] = 'YES'
        else:
            y[i] = 'NO'
    return (y)


def max_trait(x):
    x = np.array(x)
    x[x == 'NA'] = np.nan
    x = x.astype(float)
    x = np.nanmax(x, axis=0)
    x = ['NA' if np.isnan(v) else str(v).replace('.0', '') for v in x]
    return (x)



fh = open('Clinical_data.clean.TCGA-STADxxx.txt', 'wt')

survivalDa = OrderedDict()

for trt in ['chemo','radiation','targeted_molecular','days_to_last_followup','days_to_death']:
    survivalDa[trt]=[]


traits = ['gender','ethnicity','race','pathologic_stage', 
          'pathologic_T', 'pathologic_N', 'pathologic_M']


         
with open('Merged_clinical_data.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[0] == '':
            samples = [sam+'-01A' for sam in lst if sam.startswith('TCGA')]
            fh.write('\t'+'\t'.join(samples)+'\n')
        elif lst[0] == 'age_at_initial_pathologic_diagnosis':
            fh.write('age'+'\t'+'\t'.join(lst[1:])+'\n')
        elif lst[0] in traits:
            lst = [trt.replace(' ', '') for trt in lst]
            fh.write('\t'.join(lst)+'\n')
            
        elif lst[0].startswith('days_to_last_followup'):
            survivalDa['days_to_last_followup'].append(lst[1:])
            
        elif lst[0].startswith('days_to_death'):
            survivalDa['days_to_death'].append(lst[1:])
            
            
        elif lst[0].startswith('therapy_type'):
            survivalDa['chemo'].append(lst[1:])
            
        elif lst[0].startswith('radiation_therapy'):
            survivalDa['radiation'].append(lst[1:])
            
        elif lst[0].startswith('targeted_molecular_therapy'):
            survivalDa['targeted_molecular'].append(lst[1:])        

            
for ke in ['chemo','radiation','targeted_molecular']:
    fh.write(ke+'\t'+'\t'.join(therapy_info(survivalDa[ke]))+'\n')
    
for ke in ['days_to_death','days_to_last_followup']:
    fh.write(ke+'\t'+'\t'.join(max_trait(survivalDa[ke]))+'\n')


vital_status = [0 if x == 'NA' else 1 for x in max_trait(survivalDa['days_to_death'])]
fh.write('vital_status'+'\t'+'\t'.join(map(str,vital_status))+'\n')


fh.close()


/home/ruidong/Tools/anaconda3/lib/python3.5/site-packages/numpy/lib/nanfunctions.py:326: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


In [111]:
################################
### DM CG SITES

sigCG = []
with open('methylation27_test.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Tumor_Beta':
            continue
        sigCG.append(lst[0])
        
        

In [117]:
##################################################
################## INPUT FOR SURVIVAL ANALYSIS

fh = open('methylation.beta_value27_and_traits.survival.TCGA-STAD.txt', 'wt')

with open('methylation.beta_value27_and_traits.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            samples = [sam for sam in lst if re.search('-01A$', sam)]
            break
            
            
with open('Clinical_data.clean.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            samIndex = [lst.index(sam) for sam in samples if sam in lst]
            finalSam = [lst[i] for i in samIndex]
            
        trait = [lst[i] for i in samIndex]
        
        fh.write(lst[0]+'\t'+'\t'.join(trait)+'\n')
        
        
with open('methylation.beta_value27_and_traits.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            cgSamIndex = [lst.index(sam) for sam in finalSam]
            
        elif lst[0] in sigCG:
            cgValue = [lst[i] for i in cgSamIndex]
            fh.write(lst[0]+'\t'+'\t'.join(cgValue)+'\n')

fh.close()

In [4]:
####################################################################
############## survival inpput (normal)

sigCG = []
with open('methylation27_test.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Tumor_Beta':
            continue
        sigCG.append(lst[0])
        
         
traits = ['sample_type','sample_type_id','tumor_stage','tumor_grade',
          'age_at_diagnosis','days_to_death','days_to_last_follow_up',
          'vital_status','race','gender','ethnicity']

fh = open('methylation.beta_value450_and_traits.survival.TCGA-STAD.txt', 'wt')

with open('methylation.beta_value450_and_traits.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            samples = [sam for sam in lst if re.search('-01A$', sam)]
            samIndex = [lst.index(sam) for sam in samples]
            fh.write('\t'+'\t'.join(samples)+'\n')
        elif lst[0] in traits:
            trt = [lst[i] for i in samIndex]
            fh.write(lst[0]+'\t'+'\t'.join(trt)+'\n')
        elif lst[0] in sigCG:
            cgValue = [lst[i] for i in samIndex]
            fh.write(lst[0]+'\t'+'\t'.join(cgValue)+'\n')
        

fh.close()


In [5]:
###########################################
### organize methylation annotation

fh = open('methylation450_annotation.clean.txt', 'wt')
with open('methylation450_annotation.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        fh.write(lst[0]+'\t'+'\t'.join(lst[2:])+'\n')
        
fh.close()

In [18]:
methySite = OrderedDict()
with open('methylation450_annotation.clean.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        methySite[lst[0]] ='\t'.join(lst[1:])

fh = open('Package/Summary/methylation450.univariate_cox.probe.annotation.TCGA-STAD.txt', 'wt')
with open('Package/Summary/methylation450.univariate_cox.probe.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if line.startswith('coef'):
            fh.write('\t'+line+'\t'+methySite['Composite Element REF']+'\n')
            continue
            
        fh.write(line+'\t'+methySite[lst[0]]+'\n')
        
        
fh.close()

In [3]:
####################################
### for correlation analysis
symbol2ensembl = OrderedDict()
ensembl2symbol = OrderedDict()
multiGene = OrderedDict()

with open(database+'Homo_sapiens.GRCh38.86.gff3.clean.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[1] not in symbol2ensembl.keys():
            symbol2ensembl[lst[1]] = []
        symbol2ensembl[lst[1]].append(lst[0].split('.')[0])
        
        if lst[0] not in ensembl2symbol.keys():
            ensembl2symbol[lst[0].split('.')[0]] = lst[1]

for ke, val in symbol2ensembl.items():
    if len(val) > 1:
        multiGene[ke] = val

In [4]:
ensembl2symbol['ENSG00000227232']

'WASH7P'

In [27]:
methyTarget = []
i=0
with open('methylation450.cox.probe_gender_age_stage.001.annotation.TCGA-STAD.txt') as f:
    for line in f:
        line.rstrip()
        lst = line.split('\t')
        if lst[0] == '' or lst[6] == '*':
            continue
        genes = lst[9].split(';')
        for gene in genes:
            if gene not in symbol2ensembl.keys():
                print (gene)
                continue
            pair = (lst[0], symbol2ensembl[gene][0])
            if pair not in methyTarget:
                
                
                
                methyTarget.append(pair)
                
            ###### check if a gene has multiple ensembl ids
            #if gene in multiGene.keys():
            #    print (gene)
            
                
                
len(methyTarget)

RP11-944L7.5
RP11-383H13.1
RP11-383H13.1
RP11-383H13.1
RP11-383H13.1
RP11-383H13.1
RP11-383H13.1
RP11-594N15.2
AL589787.1
AC021205.1


348

In [38]:
###################################
##### overlapping samples

from collections import OrderedDict

methySamples = OrderedDict()
ovlpSamples = []

with open('methy450_STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0].endswith('01A'):
            methySamples[lst[0]] = lst[-1]

with open('RNAseq_STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] in methySamples.keys():
            ovlpSamples.append(lst[0])
        

print (len(ovlpSamples))

336


In [58]:
methyDa = OrderedDict()
rnaDa = OrderedDict()

with open('methylation.beta_value450_and_traits.survival.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            samIndex1 = [lst.index(sam) for sam in ovlpSamples]
            continue
        
        if lst[0] in [pair[0] for pair in methyTarget]:
            methyDa[lst[0]] = [lst[i] for i in samIndex1]
            

with open('RNAseq_counts_data_TCGA_STAD.normalized.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0].startswith('TCGA'):
            lst = [sam.replace('.', '-') for sam in lst]
            samIndex2 = [lst.index(sam)+1 for sam in ovlpSamples]
            continue
            
        gene = lst[0].split('.')[0]
        
        if gene in [pair[1] for pair in methyTarget]:
            rnaDa[gene] = [lst[i] for i in samIndex2]
            
            
fh1 = open('methylation450.beta.ovlp.TCGA-STAD.txt', 'wt')
fh2 = open('RNAseq_counts.normalized.ovlp.TCGA-STAD.txt', 'wt')

fh1.write('\t'+'\t'.join(ovlpSamples)+'\n')
fh2.write('\t'+'\t'.join(ovlpSamples)+'\n')

for probe, gene in methyTarget:
    fh1.write(probe+'\t'+'\t'.join(methyDa[probe])+'\n')
    fh2.write(gene+'\t'+'\t'.join(rnaDa[gene])+'\n')
    
fh1.close()
fh2.close()
    

In [74]:
fh = open('correlation_methy_rna.sigSurvMethy.clean.TCGA-STAD.txt', 'wt')
with open('correlation_methy_rna.sigSurvMethy.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 't':
            fh.write('\t'.join(['probe', 'symbol', 'geneID']+lst)+'\n')
            continue
        pair = lst[0].split('-')
        geneSymbol = [smbl for smbl in symbol2ensembl.keys() if [pair[1]]==symbol2ensembl[smbl]][0]
        fh.write('\t'.join([pair[0], geneSymbol, pair[1]]+lst[1:])+'\n')
        
fh.close()

In [96]:
##################################################
########### DE Analysis between DM groups

fh = open('DE_analysis_between_DM_groups.input.TCGA-STAD.txt', 'wt')

probes = OrderedDict()
with open('methylation450.beta.ovlp.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            fh.write(line+'\n')
            continue
        elif len(probes) < 20:
            probes[lst[0]] = lst[1:]
            
for ke, val in probes.items():
    fh.write(ke+'\t'+'\t'.join(val)+'\n')
    

with open('RNAseq.counts_only.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            samIndex = [lst.index(sam) for sam in ovlpSamples]
            continue
            
        fh.write(lst[0].split('.')[0]+'\t'+'\t'.join([lst[i] for i in samIndex])+'\n')

fh.close()

In [ ]:
####################################################################
############## survival inpput (hyper group, DEGs)

sigCG = []
with open('limma_001_2X.AKR1B1.methy_group.03.TCGA-BLCA.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'logFC':
            continue
        sigCG.append(lst[0])
        
         
traits = ['sample_type','sample_type_id','tumor_stage','tumor_grade',
          'age_at_diagnosis','days_to_death','days_to_last_follow_up',
          'vital_status','race','gender','ethnicity']

fh = open('methylation.beta_value450_and_traits.survival.TCGA-STAD.txt', 'wt')

with open('methylation.beta_value450_and_traits.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            samples = [sam for sam in lst if re.search('-01A$', sam)]
            samIndex = [lst.index(sam) for sam in samples]
            fh.write('\t'+'\t'.join(samples)+'\n')
        elif lst[0] in traits:
            trt = [lst[i] for i in samIndex]
            fh.write(lst[0]+'\t'+'\t'.join(trt)+'\n')
        elif lst[0] in sigCG:
            cgValue = [lst[i] for i in samIndex]
            fh.write(lst[0]+'\t'+'\t'.join(cgValue)+'\n')
        

fh.close()

In [101]:
chemoTherapy = OrderedDict()
with open('chemo_therapy.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[0] == '':
            continue
            
        elif lst[1] == 'NA':
            chemoTherapy[lst[0]+'-01A'] = 0
            
        else:
            chemoTherapy[lst[0]+'-01A'] = 1
            
fh = open('chemo_therapy_info.input.txt', 'wt')
with open('DE_analysis_between_DM_groups.input.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[0] == '':
            fh.write(line+'\n')
            chemo = [chemoTherapy[sam] for sam in lst if sam in chemoTherapy.keys()]
            fh.write('chemo'+'\t'+'\t'.join(map(str,chemo))+'\n')
            break
            
fh.close()
            

In [26]:
###############################################
##### annotate degs

gene = 'RP11-944L7.4'

fh = open('Run/'+gene+'/limma_001_2X.'+gene+'.risk_group.symbol.TCGA-STAD.txt', 'wt')

with open('Run/'+gene+'/limma_001_2X.'+gene+'.risk_group.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'logFC':
            fh.write('\t'+'\t'.join(['geneSymbol']+lst)+'\n')
            continue
        elif lst[0] in ensembl2symbol.keys():
            geneSymbol = ensembl2symbol[lst[0]]
            fh.write('\t'.join([lst[0],geneSymbol]+lst[1:])+'\n')
        else:
            fh.write('\t'.join([lst[0],lst[0]]+lst[1:])+'\n')
            
fh.close()

In [5]:
###############################################
##### annotate degs

gene = 'CLIP4'

fh = open('Run/'+gene+'/newResults/limma_001_2X.'+gene+'.risk_group.symbol.TCGA-STAD.txt', 'wt')

with open('Run/'+gene+'/newResults/limma_001_2X.'+gene+'.risk_group.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'logFC':
            fh.write('\t'+'\t'.join(['geneSymbol']+lst)+'\n')
            continue
        elif lst[0] in ensembl2symbol.keys():
            geneSymbol = ensembl2symbol[lst[0]]
            fh.write('\t'.join([lst[0],geneSymbol]+lst[1:])+'\n')
        else:
            fh.write('\t'.join([lst[0],lst[0]]+lst[1:])+'\n')
            
fh.close()

In [22]:
################################################
####### GSEA input

gene = 'RP11-944L7.4'

fh = open('Run/'+gene+'/'+gene+'.GSEA.expr.txt', 'wt')
ovlpGene = OrderedDict()


with open('Run/'+gene+'/'+gene+'.risk_group.genes_with_detectable_expression_levels.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if line.startswith('TCGA'):
            fh.write('NAME\t'+'\t'.join(lst)+'\n')
            
        elif lst[0] not in ensembl2symbol.keys():
            print (lst[0])
            continue
            
        else:
            geneSymbol = ensembl2symbol[lst[0]]
            if geneSymbol not in ovlpGene.keys():
                ovlpGene[geneSymbol] = 1
                fh.write(geneSymbol+'\t'+'\t'.join(lst[1:])+'\n')
                
            else:
                print (lst[0]+'\t'+geneSymbol)
        
        
fh.close()

ENSG00000155640
ENSG00000188206
ENSG00000205246
ENSG00000213971
ENSG00000225892
ENSG00000226958
ENSG00000229729
ENSG00000231025
ENSG00000241781
ENSG00000244567
ENSG00000261254
ENSG00000269131
ENSG00000269640
ENSG00000269936
ENSG00000272655	POLR2J4
ENSG00000274735
ENSG00000275530
ENSG00000276924
ENSG00000276967
ENSG00000277105
ENSG00000279978
ENSG00000280111
__no_feature
__ambiguous
__alignment_not_unique


In [50]:
########################################
###### validation dataset
valClinical = OrderedDict()
valExpr = OrderedDict()
valMethy = OrderedDict()


with open('validation/clinical.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        valClinical[lst[1]] = lst[0]
        
with open('validation/expression.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        for ke, val in valClinical.items():
            if re.search(ke,lst[1]):
                valExpr[ke] = lst[0]
                
with open('validation/methylation.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        for ke, val in valExpr.items():
            if re.search(ke+'$', lst[1]):
                valMethy[ke] = lst[0]
                
                


In [65]:
ovlpValMethy = []
ovlpValExpr = []
ovlpValClinical = []


for ke, val in valMethy.items():
    ovlpValMethy.append(val)
    ovlpValExpr.append(valExpr[ke])
    ovlpValClinical.append(valClinical[ke])
    

In [88]:
print (ovlpValExpr)

['GSM387920', 'GSM387912', 'GSM387914', 'GSM387934', 'GSM387924', 'GSM387933', 'GSM387921', 'GSM858124', 'GSM387918', 'GSM387929', 'GSM387935', 'GSM387932', 'GSM387915', 'GSM387916', 'GSM387925', 'GSM387930', 'GSM387913', 'GSM858121', 'GSM387928', 'GSM387927', 'GSM387919', 'GSM387908', 'GSM858125', 'GSM387815', 'GSM387881', 'GSM387898', 'GSM387931', 'GSM858110', 'GSM858119', 'GSM387922', 'GSM387893', 'GSM858123', 'GSM387899', 'GSM387897', 'GSM387900', 'GSM387885', 'GSM387878', 'GSM387944', 'GSM387971', 'GSM387966', 'GSM387950', 'GSM387980', 'GSM387945', 'GSM387970', 'GSM387823', 'GSM387838', 'GSM387826', 'GSM387824', 'GSM387836', 'GSM387831', 'GSM387835', 'GSM387832', 'GSM387819', 'GSM387817', 'GSM387983', 'GSM858088', 'GSM387862', 'GSM387987', 'GSM387822', 'GSM387982', 'GSM387834', 'GSM387833', 'GSM387848', 'GSM387854', 'GSM387842', 'GSM387829', 'GSM387821', 'GSM387859', 'GSM387864', 'GSM387850', 'GSM387828', 'GSM387825', 'GSM387839', 'GSM387985', 'GSM387820', 'GSM387837', 'GSM387827'

In [95]:
fh = open('validation/GPL570-55999.clean.txt', 'wt')

GPL96 = OrderedDict()
GPL570 = OrderedDict()

with open('validation/GPL96-57554.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if line.startswith('#') or lst[0]=='ID':
            continue
        if len(lst) < 10:
            continue
        GPL96[lst[0]] = lst[10].split(' /// ')
        
        
with open('validation/GPL570-55999.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if line.startswith('#') or lst[0]=='ID':
            continue
        if len(lst) < 10:
            continue
            
        GPL570[lst[0]] = lst[10].split(' /// ')
        
for ke ,val in GPL570.items():
    fh.write('\t'.join([ke]+val)+'\n')
    
fh.close()

In [87]:
fh1 = open('validation/methy.GSE30601_series_matrix.input.txt', 'wt')
fh2 = open('validation/GSE15460-GPL96_series_matrix.input.txt', 'wt')
fh3 = open('validation/GSE15460-GPL570_series_matrix.input.txt', 'wt')

with open('validation/methy.GSE30601_series_matrix.txt') as f:
    for line in f:
        line = line.rstrip()
        line = line.replace('"', '')
        lst = line.split('\t')
        
        if line.startswith('!') or line == '':
            continue
            
        elif line.startswith('ID_REF'):
            samIndex = [lst.index(sam) for sam in ovlpValMethy if sam in lst]
            fh1.write('\t'+'\t'.join([lst[i] for i in samIndex])+'\n')
        else:
            fh1.write(lst[0]+'\t'+'\t'.join([lst[i] for i in samIndex])+'\n')

        
with open('validation/GSE15460-GPL96_series_matrix.txt') as f:
    for line in f:
        line = line.rstrip()
        line = line.replace('"', '')
        lst = line.split('\t')
        
        if line.startswith('!') or line == '':
            continue
            
        elif line.startswith('ID_REF'):
            samIndex = [lst.index(sam) for sam in ovlpValExpr if sam in lst]
            fh2.write('\t'+'\t'.join([lst[i] for i in samIndex])+'\n')
        else:
            fh2.write(lst[0]+'\t'+'\t'.join([lst[i] for i in samIndex])+'\n')

        
with open('validation/GSE15460-GPL570_series_matrix.txt') as f:
    for line in f:
        line = line.rstrip()
        line = line.replace('"', '')
        lst = line.split('\t')
        
        if line.startswith('!') or line == '':
            continue
            
        elif line.startswith('ID_REF'):
            samIndex = [lst.index(sam) for sam in ovlpValExpr if sam in lst]
            fh3.write('\t'+'\t'.join([lst[i] for i in samIndex])+'\n')
        else:
            fh3.write(lst[0]+'\t'+'\t'.join([lst[i] for i in samIndex])+'\n')

        
fh1.close()
fh2.close()
fh3.close()

In [93]:
fh4 = open('validation/GSE15460_clinical.input.txt', 'wt')

with open('validation/GSE15460_clinical.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if line.startswith('GSM'):
            samIndex = [lst.index(sam) for sam in ovlpValClinical if sam in lst]
            fh4.write('\t'+'\t'.join([lst[i] for i in samIndex])+'\n')
        elif line.startswith('OS') or line.startswith('Age_at_surgery'):
            lst = [sam.split('.')[0] for sam in lst]
            fh4.write(lst[0]+'\t'+'\t'.join([lst[i] for i in samIndex])+'\n')
        else:
            fh4.write(lst[0]+'\t'+'\t'.join([lst[i] for i in samIndex])+'\n')

fh4.close()

In [ ]:
with open('methylation.beta_value27_and_traits.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            samples = [sam for sam in lst if re.search('-11A', sam)]
            
            
            indexSamType = [lst.index(samtype) for samtype in lst if samtype == 'PrimaryTumor']
            
            
        elif lst[0] == 'sample_type':
            indexSamType = [lst.index(samtype) for samtype in lst if samtype == 'PrimaryTumor']
            
            fh.write(samples[0]+'\t'+'\t'.join([samples[i] for i in indexSamType])+'\n')
            
        fh.write(lst[0]+'\t'+'\t'.join([lst[i] for i in indexSamType])+'\n')
        

fh.close()

In [ ]:
fh = open('methylation.beta_value27_and_traits.TCGA-STAD.txt', 'wt')


with open('methylation.beta_value27_and_traits.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            samples = [re.sub('-\d+A', '', sam) for sam in lst]
            
        elif lst[0] == 'sample_type':
            indexSamType = [lst.index(samtype) for samtype in lst if samtype == 'PrimaryTumor']
            
            fh.write(samples[0]+'\t'+'\t'.join([samples[i] for i in indexSamType])+'\n')
            
        fh.write(lst[0]+'\t'+'\t'.join([lst[i] for i in indexSamType])+'\n')
        

fh.close()

In [24]:
###########################################
### organize methylation annotation

fh = open('methylation27_annotation.clean.txt', 'wt')
with open('methylation27_annotation.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        fh.write(lst[0]+'\t'+'\t'.join(lst[2:])+'\n')
        
fh.close()

In [25]:
methySite = OrderedDict()
with open('methylation27_annotation.clean.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        methySite[lst[0]] ='\t'.join(lst[1:])

fh = open('methylation27_test.annotation.TCGA-STAD.txt', 'wt')
with open('methylation27_test.TCGA-STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if line.startswith('Tumor'):
            fh.write('\t'+line+'\t'+methySite['Composite Element REF']+'\n')
            continue
            
        fh.write(line+'\t'+methySite[lst[0]]+'\n')
        
        
fh.close()
        

In [3]:
from collections import OrderedDict

methy = OrderedDict()
with open('RNAseq_STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        methy[lst[0]] = lst[1]
        
        
print ('methy27:')
with open('methy27_STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] in methy.keys():
            print (line)
        

print ('methy450:')
x=0
with open('methy450_STAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] in methy.keys():
            print (line)
            x+=1
            
print (x)


methy27:
TCGA-BR-4187-01A	Primary Tumor
TCGA-BR-4191-01A	Primary Tumor
TCGA-BR-4201-01A	Primary Tumor
TCGA-BR-4253-01A	Primary Tumor
TCGA-BR-4256-01A	Primary Tumor
TCGA-BR-4257-01A	Primary Tumor
TCGA-BR-4267-01A	Primary Tumor
TCGA-BR-4279-01A	Primary Tumor
TCGA-BR-4280-01A	Primary Tumor
TCGA-BR-4294-01A	Primary Tumor
TCGA-BR-4357-01A	Primary Tumor
TCGA-BR-4361-01A	Primary Tumor
TCGA-BR-4363-01A	Primary Tumor
TCGA-BR-4366-01A	Primary Tumor
TCGA-BR-4367-01A	Primary Tumor
TCGA-BR-4368-01A	Primary Tumor
TCGA-BR-4369-01A	Primary Tumor
TCGA-BR-4370-01A	Primary Tumor
TCGA-BR-4371-01A	Primary Tumor
TCGA-CG-4301-01A	Primary Tumor
TCGA-CG-4304-01A	Primary Tumor
TCGA-CG-4305-01A	Primary Tumor
TCGA-CG-4306-01A	Primary Tumor
TCGA-CG-4436-01A	Primary Tumor
TCGA-CG-4438-01A	Primary Tumor
TCGA-CG-4440-01A	Primary Tumor
TCGA-CG-4442-01A	Primary Tumor
TCGA-CG-4443-01A	Primary Tumor
TCGA-CG-4444-01A	Primary Tumor
TCGA-CG-4460-01A	Primary Tumor
TCGA-CG-4462-01A	Primary Tumor
TCGA-CG-4465-01A	Primary Tumor